# Sentiment Analysis Using Tensorflow LSTM - CNN with Attention Layer

## Imports

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.compat.v1.keras.layers import CuDNNLSTM, Conv1D, Flatten, MaxPooling1D, Embedding, Dropout,Dense, Bidirectional, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop, Adamax , Adam
from tensorflow.keras import regularizers

from attention.layers import AttentionLayer
import tensorflow as tf
from tensorflow.python.client import device_lib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #This is for multiple print statements per cell

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

## Confirm GPU processing available

In [2]:
value = tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)
print ('***If TF can access GPU: ***\n\n',value) # MUST RETURN True IF IT CAN!!

print()
value = tf.config.list_physical_devices('GPU')
print(value)

print()
print(device_lib.list_local_devices())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
***If TF can access GPU: ***

 True

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18383731629480075645
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10510357605963443477
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5669051757671923178
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1259942707
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9524209338974458897
physical_device_desc: "device: 0, name: GeForce MX150, pci bus id: 0000:02:00.0, compute capability: 6.1"
]


## Getting data

In [3]:
df = pd.read_csv('consolidated_tweet_data-cleaned-stemmed-lemmatized.csv', sep='\t')
df

,sentiment,text
0,negative,oh no it fade away again
1,positive,bunnylak will kill me but i cant stop listen t...
2,negative,last day in cali partyin for the last time wit...
3,negative,is have a major soar throat
4,positive,my last day a 12 year old
...,...,...
1611536,negative,twisuz yeah and how did thi happen i wa updat ...
1611537,negative,smittygoali im sorri about your dog
1611538,negative,posipat im alreadi there i wish you were here
1611539,negative,is think in 12 hour ill be at the airport thi ...


In [4]:
# How much of Dataset to be used
frac = 0.2
# sample and shuffle the dataset according to the fraction choise in the line above
df1 = df.sample(frac=frac).reset_index(drop=True)
df1

,sentiment,text
0,negative,russiafi yeaah your right it holiday but it be...
1,positive,denverfoodguy thank for follow
2,positive,chk10 iamsuperbianca hi is it true about the p...
3,negative,whaaaaaaaaaaat max gt kupono sytycd
4,positive,mcjunior your welcom i do agre he is veri good...
...,...,...
322303,positive,read requiem for a dream lt3
322304,negative,tim minchin host rarraaraaarag imperson of the...
322305,positive,my elf hunter borowen just hit the one gold ma...
322306,positive,09aoc thank for the comment about ireland reco...


## Tokenization for training

In [5]:
vocabulary_size = 1000

In [6]:
tokenizer = Tokenizer(num_words=vocabulary_size, split=" ", oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'].values)

In [7]:
X = tokenizer.texts_to_sequences(df1['text'].values)
X = pad_sequences(X) # padding our text vector so they all have the same length
X[:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   1,   1,  34, 118,   6, 613,  22,   6, 100, 313,   5,
        173,  47,   2,  45, 108,  25, 782, 107,  25, 342, 445],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   1,  51,  11, 120],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   1,   1, 137,  10,   6, 474,  64,   4,   1,   1,  13,
          1, 769, 194, 473,  64,   6, 128,  11,   1,   9,  51],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   1,   1, 645,   1,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
         34, 3

### Word Embedding Matrix and weight matrix

In [ ]:
# get the embedding matrix from the embedding layer
from numpy import zeros
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
 embedding_vector = w2v.get(word)
 if embedding_vector is not None:
  embedding_matrix[i] = embedding_vector

## Model, Training and Testing

### Creating the model

In [8]:
model = Sequential([Dense()])
model.add(Embedding(vocabulary_size, 256,weights=[embedding_matrix], input_length=X.shape[1]))
model.add(Dropout(0.3))
# model.add(Conv1D(16,32,padding="same",activation="relu"))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(32,16,padding="same",activation="relu",kernel_regularizer=regularizers.l2(0.01)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.3))
# model.add(Conv1D(64,12,padding="same",activation="relu",kernel_regularizer=regularizers.l2(0.01)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.3))
# model.add(Conv1D(128,9,padding="same",activation="relu",kernel_regularizer=regularizers.l2(0.01)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.3))
# model.add(Conv1D(256,6,padding="same",activation="relu",kernel_regularizer=regularizers.l2(0.01)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
model.add(Bidirectional(CuDNNLSTM(256, return_sequences=True)))
model.add(Dropout(0.3))
model.add(AttentionLayer(name='attention'))
model.add(BatchNormalization())
model.add(Dense(2, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
# model.add(Dense(2, activation='sigmoid'))

NameError: name 'embedding_matrix' is not defined

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

### Training the model

In [ ]:
y = pd.get_dummies(df1['sentiment']).values
[print(df1['sentiment'][i], y[i]) for i in range(0,5)]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [ ]:
batch_size = 32
epochs = 10
import time
from datetime import datetime
datetime = str(datetime.now())
csv_logger = tf.keras.callbacks.CSVLogger('training'+datetime+'.log')
start = time.time()
print("started at:")
print(start)
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val,y_val), batch_size=batch_size, verbose=2, callbacks=[csv_logger])
end = time.time()
elapsed = end - start
print(elapsed/60," minutes")

Plotting training history

In [ ]:
# # Plot training & validation accuracy values
# plt.plot(history.history['accuracy'])
# # plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# # plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()


### Plotting training and validation accuracy and loss

In [ ]:
# plot train and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# plot train and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

### Saving the trained model

name of the file

In [ ]:
name = 'sentiment-analysis-trained-model'

In [ ]:
from datetime import datetime
dateTimeObj = datetime.now()
date = str(dateTimeObj.date())
time = str(dateTimeObj.time())
timestamp = date+time
punctuation = ['-',':','.']
for sign in punctuation:
    timestamp = timestamp.replace(sign,'')
print(timestamp)

In [ ]:
model.save(name+timestamp+'.h5')

### Testing the model

In [ ]:
predictions = model.predict(X_test)
[print(X_test[i], predictions[i], y_test[i]) for i in range(0, 10)]

In [ ]:
accurate_prediction_count, inaccurate_prediction_count = 0, 0
for i, prediction in enumerate(predictions):
    if np.argmax(prediction)==np.argmax(y_test[i]):
        accurate_prediction_count += 1
    else:
        inaccurate_prediction_count += 1

total_predictions = accurate_prediction_count + inaccurate_prediction_count
print('Number of predictions: ', total_predictions)
print('Number of accurate predictions: ', accurate_prediction_count)
print('Number of false predictions: ', inaccurate_prediction_count)    
print('Accuracy: ', accurate_prediction_count/total_predictions)